In [32]:
pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install sentence-transformers==1.2.0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 440.0 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-1.2.0-py3-none-any.whl size=123323 sha256=0dbd4630b225aca0cc8722bbc313473d39c3c2de43c53e16c6ae086a391ee52e
  Stored in directory: /Users/johnny/Library/Caches/pip/wheels/ef/c3/ce/320c01964e87e0487a9fba07e2d22cebc4e09e805d465667d9
Successfully built sentence-transformers
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 2.6.1
    Uninstalling sentence-transformers-2.6.1:
      Successfully uninstalled sentence-transformers-2.6.1
Note: you may need to restart the kernel to use updated packages.


In [33]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [34]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 10.0 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [35]:
pip install spacy

Note: you may need to restart the kernel to use updated packages.


In [36]:
pip install --upgrade sentence-transformers transformers

Note: you may need to restart the kernel to use updated packages.


In [37]:
pip install torch torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.0/147.0 MB 575.4 kB/s eta 0:00:0000:0100:06
  Attempting uninstall: torch
    Found existing installation: torch 2.2.2
    Uninstalling torch-2.2.2:
      Successfully uninstalled torch-2.2.2
Note: you may need to restart the kernel to use updated packages.


In [15]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("shibing624/text2vec-base-chinese")

sentence1_embeddings = model.encode(["go to the curch"])
# sentence2_embeddings = model.encode(["你是誰"])
# sentence2_embeddings = model.encode(["你過得如何"])
sentence2_embeddings = model.encode(["come on"])

similarity = util.pytorch_cos_sim(sentence1_embeddings, sentence2_embeddings)
print(similarity)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/johnny/anaconda3/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/8s/9fh2s7w53w1fks_4dpp52tyw0000gn/T/ipykernel_19161/3928678121.py", line 1, in <module>
    from sentence_transformers import SentenceTransformer, util
  File "/Users/johnny/anaconda3/lib/python3.10/site-packages/sentence_transformers/__init__.py", line 3, in <module>
    from .datasets import SentencesDataset, ParallelSentencesDataset
  File "/Users/johnny/anaconda3/lib/python3.10/site-packages/sentence_transformers/datasets/__init__.py", line 3, in <module>
    from .ParallelSentencesDataset import ParallelSentencesDataset
  File "/Users/johnny/anaconda3/lib/python3.10/site-packages/sentence_transformers/datasets/ParallelSentencesDataset.py", line 4, in <module>
    from .. import SentenceTransformer
  File "/Users/johnny/anaconda3/lib/python3.10/site-package

In [2]:
from http.server import BaseHTTPRequestHandler, HTTPServer
from sentence_transformers import SentenceTransformer, util
import mysql.connector
import json

def connect_to_database():
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="3659215fF",
        database="handle_together"
    )
    return connection

def send_json_response(self, status_code, data):
    # Convert status_code to integer
    status_code = int(status_code)

    # Set up the HTTP response
    self.send_response(status_code)
    self.send_header("Content-type", "application/json")
    self.end_headers()

    # Convert the data to JSON
    json_response = json.dumps(data)

    # Send the JSON response
    self.wfile.write(json_response.encode('utf-8'))

class WebServerHandler(BaseHTTPRequestHandler):

    def do_POST(self):       
        # Read the content length from the request headers
        content_length = int(self.headers['Content-Length'])

        # Read the JSON data from the request body
        post_data = self.rfile.read(content_length)
        json_data_list = json.loads(post_data.decode('utf-8'))

        if json_data_list:
            json_data = json_data_list[0]
            # Now json_data contains the parameters sent in the JSON format
            tag_array = json_data.get('tag_array', [])
            subject_array = json_data.get('subject_array', [])
            object_array = json_data.get('object_array', [])
            input_sentence = json_data.get('input_sentence', '')       
            public_ind = json_data.get('public_ind', '')
            command_ind = json_data.get('command_ind', '')

            print()
        # Load the Sentence Transformer model
        model = SentenceTransformer("shibing624/text2vec-base-chinese")

        # Fetch distinct subject codes from intent_type table
        connection = connect_to_database()
        cursor = connection.cursor()

        # Use parameterized query to avoid SQL injection
        # query = "SELECT tag, content FROM intent_training_data WHERE tag IN ({}) AND subject IN ({})".format(','.join(['%s' for _ in tag_array]), ','.join(['%s' for _ in subject_array]))
        if tag_array and any(tag_array):
            tag_condition = "tag IN ({})".format(','.join(['%s' for _ in tag_array]))
        else:
            tag_condition = "1=1"  # Always true if tag_array is empty or None
            
        if command_ind: 
            tag_condition2 = "AND COMMAND_IND = '"+command_ind+"'"
        else:
            tag_condition2 = ""  # Always true if tag_array is empty or None
        
        query = ("SELECT tag, content FROM intent_training_data WHERE {} AND subject IN ({}) AND object IN ({}) AND PUBLIC_IND = '"+public_ind+"'"+tag_condition2).format(tag_condition, ','.join(['%s' for _ in subject_array]) ,','.join(['%s' for _ in object_array]))
        params = tag_array + subject_array + object_array
        cursor.execute(query, params)
        rows = cursor.fetchall()
        candidate_responses_tags = [row[0] for row in rows]
        candidate_responses = [row[1] for row in rows]

        cursor.close()
        connection.close()

        # Encode input sentence and candidate responses
#         input_embedding = model.encode([input_sentence])
#         response_embeddings = model.encode(candidate_responses)

        # Calculate cosine similarity between input and responses
#         similarities = util.pytorch_cos_sim(input_embedding, response_embeddings)[0]
        

        input_embedding = model.encode([input_sentence])[0]
        response_embeddings = [model.encode([response])[0] for response in candidate_responses]

        similarities = util.pytorch_cos_sim(torch.tensor([input_embedding]), torch.tensor(response_embeddings))[0]
        similarity = util.pytorch_cos_sim(input_embedding, response_embeddings)

        # Get the index of the most similar response
        most_similar_index = similarities.argmax().item()

        # Print the most similar response
        print("Input sentence:", input_sentence)
        print("Most tag:", candidate_responses_tags[most_similar_index])
        print("Most similar response:", candidate_responses[most_similar_index])
        print("similarity:",similarity)
        result = {
            "input_sentence": input_sentence,
            "tag": candidate_responses_tags[most_similar_index],
            "content": candidate_responses[most_similar_index],
            "similarity" : round(similarity.item(),5)
        }
        send_json_response(self, '200', result)

if __name__ == '__main__':
    try:
        port = 8000
        server = HTTPServer(('localhost', port), WebServerHandler)
        server.max_body_length = 10 * 1024 * 1024  # set to 10 MB

        print(f'Server running on port {port}')
        
        server.serve_forever()
    except KeyboardInterrupt:
        print('^C received, shutting down server')
        server.socket.close()


Server running on port 8000



modules.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.0k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/409M [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

/var/folders/8s/9fh2s7w53w1fks_4dpp52tyw0000gn/T/ipykernel_79946/3504317252.py:91: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:264.)
  similarities = util.pytorch_cos_sim(torch.tensor([input_embedding]), torch.tensor(response_embeddings))[0]
----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 63265)
Traceback (most recent call last):
  File "/Users/johnny/anaconda3/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/johnny/anaconda3/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/Users/johnny/anaconda3/lib/python3.10/socketserver.py", 

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 63616)
Traceback (most recent call last):
  File "/Users/johnny/anaconda3/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/johnny/anaconda3/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/Users/johnny/anaconda3/lib/python3.10/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/johnny/anaconda3/lib/python3.10/socketserver.py", line 747, in __init__
    self.handle()
  File "/Users/johnny/anaconda3/lib/python3.10/http/server.py", line 433, in handle
    self.handle_one_request()
  File "/Users/johnny/anaconda3/lib/python3.10/http/server.py", line 421, in handle_one_request
    method()
  File "/var/folders/8s/9fh2s7w53w1fks_4dpp52tyw0000gn/T/ipykernel_79946/3

^C received, shutting down server


In [39]:
import random
from http.server import BaseHTTPRequestHandler, HTTPServer
from sentence_transformers import SentenceTransformer, util
import mysql.connector
import json
import torch

def connect_to_database():
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="3659215fF",
        database="handle_together"
    )
    return connection

def send_json_response(self, status_code, data):
    # Convert status_code to integer
    status_code = int(status_code)

    # Set up the HTTP response
    self.send_response(status_code)
    self.send_header("Content-type", "application/json")
    self.end_headers()

    # Convert the data to JSON
    json_response = json.dumps(data)

    # Send the JSON response
    self.wfile.write(json_response.encode('utf-8'))

class WebServerHandler(BaseHTTPRequestHandler):

    def do_POST(self):       
        # Read the content length from the request headers
        content_length = int(self.headers['Content-Length'])

        # Read the JSON data from the request body
        post_data = self.rfile.read(content_length)
        json_data_list = json.loads(post_data.decode('utf-8'))

        if json_data_list:
            json_data = json_data_list[0]
            # Now json_data contains the parameters sent in the JSON format
            tag_array = json_data.get('tag_array', [])
            subject_array = json_data.get('subject_array', [])
            object_array = json_data.get('object_array', [])
            input_sentence = json_data.get('input_sentence', '')       
            public_ind = json_data.get('public_ind', '')
            command_ind = json_data.get('command_ind', '')

        # Load the Sentence Transformer model
        model = SentenceTransformer("shibing624/text2vec-base-chinese")

        # Fetch distinct subject codes from intent_type table
        connection = connect_to_database()
        cursor = connection.cursor()

        # Use parameterized query to avoid SQL injection
        # query = "SELECT tag, content FROM intent_training_data WHERE tag IN ({}) AND subject IN ({})".format(','.join(['%s' for _ in tag_array]), ','.join(['%s' for _ in subject_array]))
        if tag_array and any(tag_array):
            tag_condition = "tag IN ({})".format(','.join(['%s' for _ in tag_array]))
        else:
            tag_condition = "1=1"  # Always true if tag_array is empty or None
            
        if command_ind: 
            tag_condition2 = "AND COMMAND_IND = '"+command_ind+"'"
        else:
            tag_condition2 = "" 
        query = ("SELECT tag, content FROM intent_training_data WHERE {} AND subject IN ({}) AND object IN ({}) AND PUBLIC_IND = '"+public_ind+"'"+tag_condition2).format(tag_condition, ','.join(['%s' for _ in subject_array]) ,','.join(['%s' for _ in object_array]))
        params = tag_array + subject_array + object_array
        cursor.execute(query, params)
        rows = cursor.fetchall()
        candidate_responses_tags = [row[0] for row in rows]
        candidate_responses = [row[1] for row in rows]
        # for row in rows:
        #     print(row)
        # print("SQL Query:", query)
        # print("Parameters:", params)
        cursor.close()
        connection.close()

        # Encode input sentence and candidate responses
        input_embedding = model.encode([input_sentence])
        response_embedding = model.encode([candidate_responses])

        response_embeddings = model.encode(candidate_responses)

        # Calculate cosine similarity between input and responses
        similarities = util.pytorch_cos_sim(input_embedding, response_embeddings)[0]
        
        most_similar_index = similarities.argmax().item()
        top_n_indices = torch.argsort(similarities, descending=True)[:5]

        # Randomly select 1 response from the top 5
        selected_index = random.choice(top_n_indices)
        selected_response_tag = candidate_responses_tags[selected_index]
        selected_response = candidate_responses[selected_index]
        selected_response_embedding= model.encode([selected_response])
        similarity = util.pytorch_cos_sim(input_embedding, selected_response_embedding)

        # Print the selected response
        print("Selected Response:", selected_response)
        # Print the most similar response
        print("Input sentence:", input_sentence)
        print("Most tag:", candidate_responses_tags[most_similar_index])
        print("Most similar response:", candidate_responses[most_similar_index])
        print("similarity:",similarity)
        result = {
            "input_sentence": input_sentence,
            "tag": candidate_responses_tags[most_similar_index],
            "content": selected_response,
            "similarity" : round(similarity.item(),5)
        }
        send_json_response(self, '200', result)

if __name__ == '__main__':
    try:
        port = 8000
        server = HTTPServer(('localhost', port), WebServerHandler)
        server.max_body_length = 10 * 1024 * 1024  # set to 10 MB

        print(f'Server running on port {port}')
        
        server.serve_forever()
    except KeyboardInterrupt:
        print('^C received, shutting down server')
        server.socket.close()


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/johnny/anaconda3/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/8s/9fh2s7w53w1fks_4dpp52tyw0000gn/T/ipykernel_19161/3738562682.py", line 3, in <module>
    from sentence_transformers import SentenceTransformer, util
  File "/Users/johnny/anaconda3/lib/python3.10/site-packages/sentence_transformers/__init__.py", line 3, in <module>
    from .datasets import SentencesDataset, ParallelSentencesDataset
  File "/Users/johnny/anaconda3/lib/python3.10/site-packages/sentence_transformers/datasets/__init__.py", line 3, in <module>
    from .ParallelSentencesDataset import ParallelSentencesDataset
  File "/Users/johnny/anaconda3/lib/python3.10/site-packages/sentence_transformers/datasets/ParallelSentencesDataset.py", line 4, in <module>
    from .. import SentenceTransformer
  File "/Users/johnny/anaconda3/lib/python3.10/site-package

In [7]:
import random
from http.server import BaseHTTPRequestHandler, HTTPServer
from sentence_transformers import SentenceTransformer, util
import mysql.connector
import json
import torch

def connect_to_database():
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="3659215fF",
        database="handle_together"
    )
    return connection

def send_json_response(self, status_code, data):
    # Convert status_code to integer
    status_code = int(status_code)

    # Set up the HTTP response
    self.send_response(status_code)
    self.send_header("Content-type", "application/json")
    self.end_headers()

    # Convert the data to JSON
    json_response = json.dumps(data)

    # Send the JSON response
    self.wfile.write(json_response.encode('utf-8'))

class WebServerHandler(BaseHTTPRequestHandler):

    def do_POST(self):       
        # Read the content length from the request headers
        content_length = int(self.headers['Content-Length'])

        # Read the JSON data from the request body
        post_data = self.rfile.read(content_length)
        json_data_list = json.loads(post_data.decode('utf-8'))

        if json_data_list:
            json_data = json_data_list[0]
            # Now json_data contains the parameters sent in the JSON format
            tag_array = json_data.get('tag_array', [])
            subject_array = json_data.get('subject_array', [])
            object_array = json_data.get('object_array', [])
            input_sentence = json_data.get('input_sentence', '')       
            public_ind = json_data.get('public_ind', '')
            command_ind = json_data.get('command_ind', '')

        # Load the Sentence Transformer model
        # model = SentenceTransformer("shibing624/text2vec-base-chinese")
        model = SentenceTransformer("paraphrase-MiniLM-L6-v2")

        # Fetch distinct subject codes from intent_type table
        connection = connect_to_database()
        cursor = connection.cursor()

        # Use parameterized query to avoid SQL injection
        # query = "SELECT tag, content FROM intent_training_data WHERE tag IN ({}) AND subject IN ({})".format(','.join(['%s' for  in tag_array]), ','.join(['%s' for  in subject_array]))
        if tag_array and any(tag_array):
            tag_condition = "tag IN ({})".format(','.join(['%s' for _ in tag_array]))
        else:
            tag_condition = "1=1"  # Always true if tag_array is empty or None
            
        if command_ind: 
            tag_condition2 = "AND COMMAND_IND = '"+command_ind+"'"
        else:
            tag_condition2 = "" 
        
        query = ("SELECT tag, content, subject FROM intent_training_data WHERE {} AND subject IN ({}) AND object IN ({}) AND PUBLIC_IND = '"+public_ind+"'"+tag_condition2).format(tag_condition, ','.join(['%s' for _ in subject_array]) ,','.join(['%s' for _ in object_array]))
        params = tag_array + subject_array + object_array
        cursor.execute(query, params)
        rows = cursor.fetchall()
        candidate_responses_tags = [row[0] for row in rows]
        candidate_responses = [row[1] for row in rows]
        candidate_responses_subject = [row[2] for row in rows]
        # for row in rows:
        #     print(row)
        # print("SQL Query:", query)
        # print("Parameters:", params)
        cursor.close()
        connection.close()

        # Encode input sentence and candidate responses
        input_embedding = model.encode([input_sentence])
        # response_embedding = model.encode([candidate_responses])

        response_embeddings = model.encode(candidate_responses)

        # Calculate cosine similarity between input and responses
        similarities = util.pytorch_cos_sim(input_embedding, response_embeddings)[0]
        
        most_similar_index = similarities.argmax().item()
        top_n_indices = torch.argsort(similarities, descending=True)[:5]

        # Randomly select 1 response from the top 5
        selected_index = random.choice(top_n_indices)
        selected_response_tag = candidate_responses_tags[selected_index]
        selected_response = candidate_responses[selected_index]
        selected_response_embedding= model.encode([selected_response])
        similarity = util.pytorch_cos_sim(input_embedding, selected_response_embedding)

        # Print the selected response
        print("Selected Response:", selected_response)
        # Print the most similar response
        print("Input sentence:", input_sentence)
        print("Most tag:", candidate_responses_tags[most_similar_index])
        print("Most similar response:", candidate_responses[most_similar_index])
        print("subject response:", candidate_responses_subject[most_similar_index])
        
        print("similarity:",similarity)
        result = {
            "input_sentence": input_sentence,
            "tag": candidate_responses_tags[most_similar_index],
            "content": selected_response,
            "sim_content" : candidate_responses[most_similar_index],
            "similarity" : round(similarity.item(),5),
            "subject" : candidate_responses_subject[most_similar_index]
        }
        send_json_response(self, '200', result)

if __name__ == '__main__':
    try:
        port = 8000
        server = HTTPServer(('localhost', port), WebServerHandler)
        server.max_body_length = 10 * 1024 * 1024  # set to 10 MB

        print(f'Server running on port {port}')
        
        server.serve_forever()
    except KeyboardInterrupt:
        print('^C received, shutting down server')
        server.socket.close()

Server running on port 8000
Selected Response: Hurry up
Input sentence: hurry
Most tag: PUSH
Most similar response: Hurry up
subject response: HOST
similarity: tensor([[0.9038]])


127.0.0.1 - - [17/Apr/2024 16:51:08] "POST / HTTP/1.1" 200 -
----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 53557)
Traceback (most recent call last):
  File "/Users/johnny/anaconda3/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/johnny/anaconda3/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/Users/johnny/anaconda3/lib/python3.10/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/johnny/anaconda3/lib/python3.10/socketserver.py", line 747, in __init__
    self.handle()
  File "/Users/johnny/anaconda3/lib/python3.10/http/server.py", line 433, in handle
    self.handle_one_request()
  File "/Users/johnny/anaconda3/lib/python3.10/http/server.py", line 421, in handle_one_request
    method()
  File "/var/

^C received, shutting down server


In [26]:
import spacy

def extract_location_with_spacy(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)

    # Extract locations from the spaCy document
    locations = [ent.text for ent in doc.ents if ent.label_ == "ORG"]

    return locations

text = "I work in Kooppi , this is my favorite city."
result = extract_location_with_spacy(text)
print(result)


[]


In [30]:
pip install --upgrade transformers

Note: you may need to restart the kernel to use updated packages.


In [42]:
pip list

Package                       Version
----------------------------- -----------
absl-py                       1.3.0
aiohttp                       3.8.3
aiosignal                     1.2.0
alabaster                     0.7.12
anaconda-client               1.11.1
anaconda-navigator            2.4.0
anaconda-project              0.11.1
annotated-types               0.6.0
anyio                         3.5.0
appdirs                       1.4.4
applaunchservices             0.3.0
appnope                       0.1.2
appscript                     1.1.2
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
array-record                  0.4.1
arrow                         1.2.3
astroid                       2.14.2
astropy                       5.1
asttokens                     2.0.5
astunparse                    1.6.3
async-timeout                 4.0.2
atomicwrites                  1.4.0
attrs                         22.1.0
Automat                       20.2.0
autopep8      

In [28]:
pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 1.2 MB/s eta 0:00:0000:0100:010m
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2
Note: you may need to restart the kernel to use updated packages.


In [41]:
from sentence_transformers import SentenceTransformer, util


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/johnny/anaconda3/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/8s/9fh2s7w53w1fks_4dpp52tyw0000gn/T/ipykernel_19161/2887606915.py", line 1, in <module>
    from sentence_transformers import SentenceTransformer, util
  File "/Users/johnny/anaconda3/lib/python3.10/site-packages/sentence_transformers/__init__.py", line 3, in <module>
    from .datasets import SentencesDataset, ParallelSentencesDataset
  File "/Users/johnny/anaconda3/lib/python3.10/site-packages/sentence_transformers/datasets/__init__.py", line 3, in <module>
    from .ParallelSentencesDataset import ParallelSentencesDataset
  File "/Users/johnny/anaconda3/lib/python3.10/site-packages/sentence_transformers/datasets/ParallelSentencesDataset.py", line 4, in <module>
    from .. import SentenceTransformer
  File "/Users/johnny/anaconda3/lib/python3.10/site-package

In [ ]:
import random
from http.server import BaseHTTPRequestHandler, HTTPServer
from sentence_transformers import SentenceTransformer, util
import mysql.connector
import json
import torch

def connect_to_database():
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="3659215fF",
        database="handle_together"
    )
    return connection

def send_json_response(self, status_code, data):
    # Convert status_code to integer
    status_code = int(status_code)

    # Set up the HTTP response
    self.send_response(status_code)
    self.send_header("Content-type", "application/json")
    self.end_headers()

    # Convert the data to JSON
    json_response = json.dumps(data)

    # Send the JSON response
    self.wfile.write(json_response.encode('utf-8'))

class WebServerHandler(BaseHTTPRequestHandler):

    def do_POST(self):       
        # Read the content length from the request headers
        content_length = int(self.headers['Content-Length'])

        # Read the JSON data from the request body
        post_data = self.rfile.read(content_length)
        json_data_list = json.loads(post_data.decode('utf-8'))

        if json_data_list:
            json_data = json_data_list[0]
            # Now json_data contains the parameters sent in the JSON format
            tag_array = json_data.get('tag_array', [])
            subject_array = json_data.get('subject_array', [])
            object_array = json_data.get('object_array', [])
            input_sentence = json_data.get('input_sentence', '')       
            public_ind = json_data.get('public_ind', '')
            command_ind = json_data.get('command_ind', '')

        # Load the Sentence Transformer model
        # model = SentenceTransformer("shibing624/text2vec-base-chinese")
        model = SentenceTransformer("paraphrase-MiniLM-L6-v2")

        # Fetch distinct subject codes from intent_type table
        connection = connect_to_database()
        cursor = connection.cursor()

        # Use parameterized query to avoid SQL injection
        if tag_array and any(tag_array):
            tag_condition = "tag IN ({})".format(','.join(['%s' for _ in tag_array]))
        else:
            tag_condition = "1=1"  # Always true if tag_array is empty or None
            
        if command_ind: 
            tag_condition2 = "AND COMMAND_IND = '"+command_ind+"'"
        else:
            tag_condition2 = "" 
        
        query = ("SELECT tag, content, subject FROM intent_training_data WHERE {} AND subject IN ({}) AND object IN ({}) AND PUBLIC_IND = '"+public_ind+"'"+tag_condition2).format(tag_condition, ','.join(['%s' for _ in subject_array]) ,','.join(['%s' for _ in object_array]))
        params = tag_array + subject_array + object_array
        cursor.execute(query, params)
        rows = cursor.fetchall()
        candidate_responses_tags = [row[0] for row in rows]
        candidate_responses = [row[1] for row in rows]
        candidate_responses_subject = [row[2] for row in rows]
        cursor.close()
        connection.close()
        print('1')
        # Handle the case when no candidate responses are found
        if not candidate_responses:
            print('2')

            result = {
                "input_sentence": input_sentence,
                "tag": "",
                "content": "😊",
                "sim_content": "",
                "similarity": 0.0,
                "subject": ""
            }
            send_json_response(self, '200', result)
            return

        # Encode input sentence and candidate responses
        input_embedding = model.encode([input_sentence])
        response_embeddings = model.encode(candidate_responses)

        # Calculate cosine similarity between input and responses
        similarities = util.pytorch_cos_sim(input_embedding, response_embeddings)[0]
        
        most_similar_index = similarities.argmax().item()
        top_n_indices = torch.argsort(similarities, descending=True)[:5]

        # Randomly select 1 response from the top 5
        selected_index = random.choice(top_n_indices)
        selected_response_tag = candidate_responses_tags[selected_index]
        selected_response = candidate_responses[selected_index]
        selected_response_embedding= model.encode([selected_response])
        similarity = util.pytorch_cos_sim(input_embedding, selected_response_embedding)

        # Print the selected response
        print("Selected Response:", selected_response)
        # Print the most similar response
        print("Input sentence:", input_sentence)
        print("Most tag:", candidate_responses_tags[most_similar_index])
        print("Most similar response:", candidate_responses[most_similar_index])
        print("subject response:", candidate_responses_subject[most_similar_index])
        
        print("similarity:",similarity)
        result = {
            "input_sentence": input_sentence,
            "tag": candidate_responses_tags[most_similar_index],
            "content": selected_response,
            "sim_content" : candidate_responses[most_similar_index],
            "similarity" : round(similarity.item(),5),
            "subject" : candidate_responses_subject[most_similar_index]
        }
        send_json_response(self, '200', result)

if __name__ == '__main__':
    try:
        port = 8000
        server = HTTPServer(('localhost', port), WebServerHandler)
        server.max_body_length = 10 * 1024 * 1024  # set to 10 MB

        print(f'Server running on port {port}')
        
        server.serve_forever()
    except KeyboardInterrupt:
        print('^C received, shutting down server')
        server.socket.close()

Server running on port 8000
1
Selected Response: Let's pick up the pace, time's ticking.

Input sentence: this is a day
Most tag: PUSH
Most similar response: Accelerate, my friend. We haven't got all day.

subject response: BOT_MAFIA
similarity: tensor([[0.3990]])


127.0.0.1 - - [19/Apr/2024 16:43:25] "POST / HTTP/1.1" 200 -


1
Selected Response: Sure thing, I'm on board with that.

Input sentence: this is a day
Most tag: APPROVE
Most similar response: Alright, I'm giving you the green light.

subject response: BOT_MAFIA
similarity: tensor([[0.1565]])


127.0.0.1 - - [19/Apr/2024 16:43:27] "POST / HTTP/1.1" 200 -


1
Selected Response: Hi
Input sentence: hi lou
Most tag: CALL
Most similar response: Hi
subject response: BOT002
similarity: tensor([[0.7911]])


...

127.0.0.1 - - [19/Apr/2024 16:49:44] "POST / HTTP/1.1" 200 -


1
Selected Response: Hey.
Input sentence: huh
Most tag: APPROVE
Most similar response: Yes
subject response: HOST
similarity: tensor([[0.7330]])


127.0.0.1 - - [19/Apr/2024 16:50:18] "POST / HTTP/1.1" 200 -


1
Selected Response: 你真是个爷们儿，继续保持！

Input sentence: huh
Most tag: REWARD
Most similar response: 很牛，真不错！

subject response: BOT_BRO_CHN
similarity: tensor([[0.3092]])


127.0.0.1 - - [19/Apr/2024 16:50:20] "POST / HTTP/1.1" 200 -
